# Detector resolution
As you know, detectors are not perfect and do not provide an infinitely precise measurement. Instead they have resolutions that may be a constant value, or may depend on the measured value itself.
In this notebook we will explore the energy resolution of the LKr detector, and the time and momentum resolution of the Spectrometer.

The resolution is the difference that we may find between the measured value and a reference value (the "real" value). Unless working with simulation, we usually cannot know the exact real value. So we need to find either a reference value which was measured with a resolution that was much smaller than the resolution that we are trying to measure. This would give us an "exact" value for all practical purposes. Alternatively we can measure the resolution with respect to another measurement for which the resolution is already well known, or with respect to another measurement provided by the same detector (if this is practical). For these two last cases, the measured resolution will be the convolution of both measurements but the resolution of the tested detector can be obtained by deconvolution. 

In [ ]:
# As usual let's import all we need
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from na62 import prepare, hlf, extract, constants, stats

In [ ]:
# Then we load the data
data, _ = prepare.import_root_files(["data/run12450.root"])

# LKr energy resolution
To measure the LKr energy resolution we can use one of the principles that we have seen in the PID notebook. We know that in almost 100% of the cases, electrons will leave all their energy inside the LKr. As a consequence if we can isolate a pure sample of electrons and knowing their energy (momenta), we have a reference for the energy that is supposed to be measured in the LKr. This can easily be provided by the Ke3 decay.

In [ ]:
# select ke3
ke3 = data.loc[data["event_type"]==constants.event_type_map["ke3"]]

In [ ]:
e = extract.track(ke3, 1)
e_energy = np.sqrt(e["momentum_mag"]**2 + constants.electron_mass**2)
delta_e = e["lkr_energy"] - e_energy

In [ ]:
cut = pd.cut(e_energy, np.arange(0,80000, 1000), labels=False)
momentum = []
values = []
err = []
plot = False
for binID in cut.value_counts().sort_index().index:
    delta_e_bin = delta_e.loc[cut==binID]
    if(len(delta_e_bin)<=100): # Not enough stat in the bin
        continue
    if binID == 20:
        plot = True
    fitr = stats.fit_gaussian(delta_e_bin, bins=100, display_range=(-5000, 5000), plot=plot)
    momentum.append(binID*1000 + 500)
    values.append(fitr.params["sigma"].value)
    err.append(fitr.params["sigma"].stderr)
    if plot:
        plt.title("Energy difference between LKr and Spectrometer\nfor Ke3 electron in the bin 20 GeV < $E_e$ < 21  GeV")
        plt.xlabel("$\Delta E$ [MeV]")
        display(fitr)
    plot = False

momentum = np.array(momentum)
values = np.array(values)
err = np.array(err)

In [ ]:
plt.errorbar(momentum, values/momentum, yerr=err/momentum, capsize=2)

# Spectrometer momentum resolution

In [ ]:
k3pi = data.loc[data["event_type"]==constants.event_type_map["k3pi"]]

In [ ]:
t1 = hlf.set_mass(extract.track(k3pi, 1), constants.pion_charged_mass)
t2 = hlf.set_mass(extract.track(k3pi, 2), constants.pion_charged_mass)
t3 = hlf.set_mass(extract.track(k3pi, 3), constants.pion_charged_mass)
beam = extract.get_beam(k3pi)
expected_t1 = hlf.three_vectors_sum([beam, hlf.three_vector_invert(t2), hlf.three_vector_invert(t3)])
cut = pd.cut(expected_t1["momentum_mag"], np.arange(0,80000, 1000), labels=False)
momentum = []
values = []
err = []
plot = False
for binID in cut.value_counts().sort_index().index:
    delta_p = expected_t1.loc[cut==binID]["momentum_mag"] - t1.loc[cut==binID]["momentum_mag"]
    if(len(delta_p)<=100):
        continue
    if binID == 20:
        plot = True
    fitr = stats.fit_gaussian(delta_p, bins=100, display_range=(-3000, 3000), plot=plot)
    momentum.append(binID*1000 + 500)
    values.append(fitr.params["sigma"].value)
    err.append(fitr.params["sigma"].stderr)
    if plot:
        plt.title("Momentum difference between expected and measured momentum\nfor a K3pi pion in the bin 20 GeV < $E_e$ < 21  GeV")
        plt.xlabel("$\Delta p$ [MeV]")
        display(fitr)
    plot = False

momentum = np.array(momentum)
values = np.array(values)
err = np.array(err)

In [ ]:
plt.errorbar(momentum, values/momentum, yerr=err/momentum, capsize=2)